In [3]:
import pandas as pd
import xlrd #pip install xlrd

#运行单元格 ctrl + alt + enter


In [4]:
#读取整个工作表数据，默认读取第一个工作表
df_beijing = pd.read_excel('beijingchanliang.xls')#相对路径，放在跟代码同一个文件夹里

#打印前几行数据
print(df_beijing.head())

     时间   品类  指标  地区 周期  单位        数值
0  2021   小麦  产量  北京  年  万吨  6.849794
1  2021  冬小麦  产量  北京  年  万吨  6.800000
2  2021  春小麦  产量  北京  年  万吨  0.000000
3  2020   小麦  产量  北京  年  万吨  4.618499
4  2020  冬小麦  产量  北京  年  万吨  4.500000


In [6]:
#获取列数据
column_data = df_beijing['品类']
print(column_data)

0     小麦
1    冬小麦
2    春小麦
3     小麦
4    冬小麦
5    春小麦
6     小麦
7    冬小麦
8    春小麦
Name: 品类, dtype: object


In [16]:
#筛选(剔除产量为0的数据)
filtered_data = df_beijing[df_beijing['数值']>0]

print(filtered_data)

     时间   品类  指标  地区 周期  单位        数值
0  2021   小麦  产量  北京  年  万吨  6.849794
1  2021  冬小麦  产量  北京  年  万吨  6.800000
3  2020   小麦  产量  北京  年  万吨  4.618499
4  2020  冬小麦  产量  北京  年  万吨  4.500000
5  2020  春小麦  产量  北京  年  万吨  0.100000
6  2019   小麦  产量  北京  年  万吨  4.410597
7  2019  冬小麦  产量  北京  年  万吨  4.400000


In [10]:
summary=df_beijing.describe()
print(summary)

                时间        数值
count     9.000000  9.000000
mean   2020.000000  3.519877
std       0.866025  2.785179
min    2019.000000  0.000000
25%    2019.000000  0.100000
50%    2020.000000  4.410597
75%    2021.000000  4.618499
max    2021.000000  6.849794


count()：计算非缺失值的数量。
sum()：计算所有值的和。
mean()：计算平均值。
median()：计算中位数。
std()：计算标准差。
var()：计算方差。
min()：找到最小值。
max()：找到最大值。
quantile(q)：计算给定分位数（0到1之间）的值


In [18]:
#数据集制作
#1.分割特征和标签 （最后一列是标签列）
features  = filtered_data.iloc[:, :-1] #所有列除了最后一列
print(features)

labels = filtered_data['数值']
print(labels)

     时间   品类  指标  地区 周期  单位
0  2021   小麦  产量  北京  年  万吨
1  2021  冬小麦  产量  北京  年  万吨
3  2020   小麦  产量  北京  年  万吨
4  2020  冬小麦  产量  北京  年  万吨
5  2020  春小麦  产量  北京  年  万吨
6  2019   小麦  产量  北京  年  万吨
7  2019  冬小麦  产量  北京  年  万吨
0    6.849794
1    6.800000
3    4.618499
4    4.500000
5    0.100000
6    4.410597
7    4.400000
Name: 数值, dtype: float64


In [30]:
#文本特征处理
#1.标签编码 转换为整数值
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()#实例化一个labelencoder
encoded_variety = label_encoder.fit_transform(features['品类'])
encoded_region = label_encoder.fit_transform(features['地区'])
print(encoded_variety)
print(encoded_region)

#替换原有的文本列（品类、地区）
features['品类'] = encoded_variety
features['地区'] = encoded_region

#删除不必要的列
features=features.drop('指标',axis=1)
features=features.drop('周期',axis=1)
features=features.drop('单位',axis=1)
print(features)


[1 0 1 0 2 1 0]
[0 0 0 0 0 0 0]
     时间  品类  地区
0  2021   1   0
1  2021   0   0
3  2020   1   0
4  2020   0   0
5  2020   2   0
6  2019   1   0
7  2019   0   0


In [26]:
#2.独热编码
one_hot_variety = pd.get_dummies(features['品类'], prefix='Variety')
one_hot_region = pd.get_dummies(features['地区'], prefix='Region')
print(one_hot_variety)

   Variety_冬小麦  Variety_小麦  Variety_春小麦
0            0           1            0
1            1           0            0
3            0           1            0
4            1           0            0
5            0           0            1
6            0           1            0
7            1           0            0


文本向量化：如果你有更复杂的文本特征，如商品描述或地区描述，你可能需要将它们转换成数值特征。常用的方法包括词袋模型、TF-IDF表示和词嵌入（如Word2Vec、GloVe）等。这些方法将文本转换为向量表示，以便模型可以处理。

In [ ]:
# #3.文本向量化
# from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# # 词袋模型
# count_vectorizer = CountVectorizer()
# count_features = count_vectorizer.fit_transform(features['Description'])

# # TF-IDF表示
# tfidf_vectorizer = TfidfVectorizer()
# tfidf_features = tfidf_vectorizer.fit_transform(features['Description'])


In [31]:
#合并特征和标签
final_data = pd.concat([features,labels],axis=1)#axis=1 沿着列的方向进行操作
print(final_data)

#保存数据集为csv格式
final_data.to_csv('processed_data.csv',index=False)#不包含行索引

     时间  品类  地区        数值
0  2021   1   0  6.849794
1  2021   0   0  6.800000
3  2020   1   0  4.618499
4  2020   0   0  4.500000
5  2020   2   0  0.100000
6  2019   1   0  4.410597
7  2019   0   0  4.400000
